# Knowledge Distillation on MNIST
Knowledge distillation is the process of transferring the higher performance of a more expensive model to a smaller one.  In this notebook, we will explore performing this process on MNIST.  To begin with, I have provided access to pre-trained model that is large, but performant.  The exact architecture is not relevant (although you can inspect this easily if you wish).  It is straightforward to load in pytorch with

In [21]:
import torch
device = 'cpu'

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(28**2,800)
        self.l2 = torch.nn.Linear(800,800)
        self.l3 = torch.nn.Linear(800,10)
        self.dropout2 = torch.nn.Dropout(0.5)
        self.dropout3 = torch.nn.Dropout(0.5)

    def forward(self, x):
        x = self.l1(x)
        x = torch.relu(x)
        x = self.dropout2(x)
        x = self.l2(x)
        x = torch.relu(x)
        x = self.dropout3(x)
        x = self.l3(x)
        return x

big_model = torch.load('pretrained_model.pt').to(device)

First, let's establish the baseline performance of the big model on the MNIST test set.  Of course we'll need acces to the MNIST test set to do this.  At the same time, let's also get our transfer set, which in this case will be a $n=10$k subset of the full MNIST training set (using a subset is helpful for speeding up training of distilled models, and also helps showcase some of the improved performance due to model distillation).   

In [22]:
from torchvision import transforms, datasets
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
    ])

dataset_train = datasets.MNIST('./data', train=True, download=True, transform=transform)

dataset_test = datasets.MNIST('../data', train=False, download=True, transform=transform)

# This is a useful function that I didn't know about before
first_10k = list(range(0, 10000))
dataset_transfer = torch.utils.data.Subset(dataset_train, first_10k)

batch_size = 32
num_workers = 4
transfer_loader = torch.utils.data.DataLoader(dataset_transfer,batch_size=batch_size,num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(dataset_test,batch_size=batch_size,num_workers=num_workers)

Here's a function that runs the big model in test mode and provides the number of correct examples

In [23]:
def test(model,test_loader):
    correct = 0
    counter = 0
    model.eval()
    with torch.no_grad():
        for data,target in test_loader:
            data, target = data.to(device), target.to(device)
            data = data.reshape(data.shape[0],-1)
            logits = model(data)
            pred = logits.argmax(dim=1,keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            counter += batch_size
    return correct

test(big_model,test_loader)

9833

We find that the big model gets 167 examples wrong (not quite as good as the Hinton paper, but who cares). 

Now we would like to perform knowledge distillation by training a smaller model to approximate the larger model's performance on the transfer set.  First, let's build a smaller model.  You may use whatever architecture you choose, but I found that using two hidden layers, each with 200 units along with ReLU activations (and no regularization at all) worked fine.

In [24]:
class SmallNet(torch.nn.Module):
    def __init__(self):
        super(SmallNet, self).__init__()
        self.l1 = torch.nn.Linear(28**2,200)
        self.l2 = torch.nn.Linear(200,200)
        self.l3 = torch.nn.Linear(200,200)
        self.l4 = torch.nn.Linear(200,10)

    def forward(self, x):
        x = self.l1(x)
        x = torch.relu(x)
        x = self.l2(x)
        x = torch.relu(x)
        x = self.l3(x)
        x = torch.relu(x)
        x = self.l4(x)
#         x = torch.softmax(x)
        return x

**To establish a baseline performance level, train the small model on the transfer set**  

In [25]:
T = 1
# I'm giving you this training function: you'll need to modify it below to do knowledge distillation
def train(model,train_loader,n_epochs):
    optimizer = torch.optim.Adam(model.parameters(),1e-3)
    loss_fn = torch.nn.CrossEntropyLoss()
    model.train()
    for epoch in range(n_epochs):
        avg_l = 0.0
        counter = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            data = data.reshape(data.shape[0],-1)
            optimizer.zero_grad()
            logits = model(data)
            L = loss_fn(logits,target)
            L.backward()
            optimizer.step()
            with torch.no_grad():
                avg_l += L
                counter += 1
        print(epoch,avg_l/counter)


In [26]:
small_model = SmallNet()
small_model.to(device)

train(small_model,transfer_loader,50)

0 tensor(0.4893)
1 tensor(0.2020)
2 tensor(0.1235)
3 tensor(0.0972)
4 tensor(0.0788)
5 tensor(0.0597)
6 tensor(0.0573)
7 tensor(0.0469)
8 tensor(0.0341)
9 tensor(0.0468)
10 tensor(0.0298)
11 tensor(0.0242)
12 tensor(0.0405)
13 tensor(0.0234)
14 tensor(0.0374)
15 tensor(0.0279)
16 tensor(0.0195)
17 tensor(0.0259)
18 tensor(0.0265)
19 tensor(0.0232)
20 tensor(0.0250)
21 tensor(0.0098)
22 tensor(0.0178)
23 tensor(0.0229)
24 tensor(0.0096)
25 tensor(0.0253)
26 tensor(0.0141)
27 tensor(0.0139)
28 tensor(0.0269)
29 tensor(0.0141)
30 tensor(0.0084)
31 tensor(0.0086)
32 tensor(0.0071)
33 tensor(0.0208)
34 tensor(0.0182)
35 tensor(0.0103)
36 tensor(0.0272)
37 tensor(0.0154)
38 tensor(0.0041)
39 tensor(0.0109)
40 tensor(0.0035)
41 tensor(0.0018)
42 tensor(0.0071)
43 tensor(0.0032)
44 tensor(0.0230)
45 tensor(0.0476)
46 tensor(0.0109)
47 tensor(0.0094)
48 tensor(0.0087)
49 tensor(0.0146)


**Evaluate the small model on the test set, and comment on its accuracy relative to the big model.**  As you might expect, the performance is relatively worse.  

In [27]:
test(small_model,test_loader)

9551

**The primary task of this notebook is now as follows: create a new training function similar to "train" above, but instead called "distill".**  "distill" should perform knowledge distillation as outlined in this week's paper.  It should accept a few additional arguments compared to train, namely the big model, the temperature hyperparameter, and a hyperparameter $\alpha$ that weights the relative magnitude of the soft target loss and the hard target loss.

In [ ]:
from torch impo

In [28]:
import torch.nn.functional as F

In [30]:
distilled_model = SmallNet()
distilled_model.to(device)

# The body of this method is currently copied verbatim from the train method above: 
# you will need to modify it to utilize the big_model, temperature, and alpha values 
# to perform knowledge distillation

def distillation(y, labels, teacher_scores, T, alpha):
    return torch.nn.KLDivLoss()(F.log_softmax(y/T), F.softmax(teacher_scores/T)) * (T*T * 2.0 * alpha) + F.cross_entropy(y, labels) * (1. - alpha)


def distill(small_model,big_model,T,alpha,transfer_loader,n_epochs):
    optimizer = torch.optim.Adam(small_model.parameters(),1e-3)
    # I had trouble getting CE to work with my loss, so found a doc online suggesting KL for loss here
    loss_fn = torch.nn.KLDivLoss()
    small_model.train()
    big_model.eval()
    for epoch in range(n_epochs):
        avg_l = 0.0
        counter = 0
        for batch_idx, (data, target) in enumerate(transfer_loader):
            data, target = data.to(device), target.to(device)
            data = data.reshape(data.shape[0],-1)
            optimizer.zero_grad()
            logits = small_model(data)
            big_logits = big_model(data).detach()
            L = distillation(logits, target, big_logits, T=T, alpha=alpha)
            L = loss_fn(F.log_softmax(logits/T), F.softmax(big_logits/T)) * (T*T * 2.0 * alpha) + F.cross_entropy(logits, target) * (1. - alpha)
            L.backward()
            optimizer.step()
            with torch.no_grad():
                avg_l += L
                counter += 1
        print(epoch,avg_l/counter)
        
distill(distilled_model,big_model,T=2,alpha=0.7,transfer_loader=transfer_loader,n_epochs=50)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:2611: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. 

0 tensor(0.4334)
1 tensor(0.1782)
2 tensor(0.1145)
3 tensor(0.0802)
4 tensor(0.0602)
5 tensor(0.0494)
6 tensor(0.0396)
7 tensor(0.0299)
8 tensor(0.0409)
9 tensor(0.0279)
10 tensor(0.0302)
11 tensor(0.0285)
12 tensor(0.0187)
13 tensor(0.0207)
14 tensor(0.0234)
15 tensor(0.0174)
16 tensor(0.0176)
17 tensor(0.0294)
18 tensor(0.0228)
19 tensor(0.0157)
20 tensor(0.0131)
21 tensor(0.0130)
22 tensor(0.0169)
23 tensor(0.0168)
24 tensor(0.0125)
25 tensor(0.0145)
26 tensor(0.0205)
27 tensor(0.0113)
28 tensor(0.0079)
29 tensor(0.0127)
30 tensor(0.0104)
31 tensor(0.0071)
32 tensor(0.0056)
33 tensor(0.0054)
34 tensor(0.0072)
35 tensor(0.0444)
36 tensor(0.0245)
37 tensor(0.0108)
38 tensor(0.0061)
39 tensor(0.0048)
40 tensor(0.0039)
41 tensor(0.0037)
42 tensor(0.0036)
43 tensor(0.0036)
44 tensor(0.0049)
45 tensor(0.0455)
46 tensor(0.0246)
47 tensor(0.0108)
48 tensor(0.0056)
49 tensor(0.0043)


**Finally, test your distilled model (on the test set) and describe how it performs relative to both big and small models.**

In [31]:
test(distilled_model,test_loader)

9664

Re